In [1]:
import os
os.environ['SPARK_HOME'] = r'C:\Users\Marcos\Documents\Spark'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [8]:
from lorem_text import lorem
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

In [7]:
par_1 = lorem.paragraph()
par_2 = lorem.paragraph()
text = par_1 + '\n' + par_2

with open('data/text.txt', 'w') as txt:
    txt.write(text)

In [9]:
spark = SparkSession.builder.appName('DataFrame-demo').getOrCreate()

In [10]:
rdd = spark.sparkContext.textFile('data/text.txt')

In [30]:
result_rdd = rdd.flatMap(lambda line: line.split(' ')) \
                .map(lambda word: (word, 1)) \
                .reduceByKey(lambda a, b: a + b) \
                .sortBy(lambda x: x[1], ascending=False)

In [31]:
result_rdd.take(10)

[('ex', 4),
 ('modi', 4),
 ('labore', 4),
 ('autem', 4),
 ('numquam', 4),
 ('eius', 3),
 ('nemo', 3),
 ('iste', 3),
 ('voluptate', 3),
 ('magnam', 3)]

In [32]:
df = spark.read.text('data/text.txt')

In [57]:
result_df = df.selectExpr("explode(split(value, ' ')) as word") \
              .groupby('word').count().orderBy(desc('count'))

In [58]:
result_df.take(10)

[Row(word='numquam', count=4),
 Row(word='ex', count=4),
 Row(word='labore', count=4),
 Row(word='modi', count=4),
 Row(word='autem', count=4),
 Row(word='ea', count=3),
 Row(word='vel', count=3),
 Row(word='voluptate', count=3),
 Row(word='adipisci', count=3),
 Row(word='sit', count=3)]